In [1]:
import pandas as pd
import json
import random

def generate_protocol(input_csv, protocol_name, protocol_type):
    # protocol type: 
    #       0 = for gestures 
    #       1 = for reaction time test

    # Read CSV into a DataFrame
    df = pd.read_csv(input_csv)


    # Initialize common JSON structure
    json_data = {
        "name": protocol_name,
        "parameters": [],
        "active": True,
        "classes": [],
        "randomClassOrder": randomClassOrder,
        "preamble": "",
        "epilogue": "",
        "group": "",
        "disconnectDeviceOnFinish": disconnectDeviceOnFinish
    }

    # Iterate through rows in the CSV
    for index, row in df.iterrows():
        # Extract relevant information from the CSV row
        class_name = row['class_name']
        class_preamble = row['class_preamble']
        class_epilogue = row['class_epilogue']

        if protocol_type == 0:
            inactive_duration = row['inactive_duration']
            active_duration = row['active_duration']
            activeClass = row['activeClass_name']
            repetitions = int(row['repetitions'])

            # Create the class structure for standard protocol
            class_structure = {
                "class": class_name,
                "intervals": [
                    {"label": "inactive", "durationInMillis": inactive_duration},
                    {"label": activeClass, "durationInMillis": active_duration},
                    {"label": "inactive", "durationInMillis": inactive_duration}
                ],
                "repetitions": repetitions,
                "preamble": class_preamble,
                "epilogue": class_epilogue
            }
        elif protocol_type == 1:
            total_duration = int(row['total_duration'])
            stimulus_duration = int(row['stimulus_duration'])
            max_isi = int(row['max_isi'])
            min_isi = int(row['min_isi'])
            r_led = int(row['r_led'])
            g_led = int(row['g_led'])
            b_led = int(row['b_led'])
            led_color = [r_led, g_led, b_led]
            intensity = int(row['intensity'])

            # Generate random isi 
            num_intervals = int(total_duration / (stimulus_duration + (max_isi + min_isi) / 2))
            intervals = []
            for _ in range(num_intervals):
                isi = random.randint(min_isi, max_isi)
                intervals.append(isi)

            # Create the class structure for RT protocol
            class_structure = {
                "class": class_name,
                "intervals": [],
                "preamble": class_preamble,
                "epilogue": class_epilogue,
                "type": "REACTION_TIME"
            }

            class_data = class_structure

            for i, duration in enumerate(intervals):
                # Unlabeled interval
                class_data["intervals"].append({
                    "durationInMillis": duration,
                    "ledOptions": {
                        "enabled": False
                    }
                })

                # STIM interval
                class_data["intervals"].append({
                    "label": "STIM",
                    "durationInMillis": stimulus_duration,
                    "ledOptions": {
                        "color": {
                            "red": led_color[0],
                            "green": led_color[1],
                            "blue": led_color[2]
                        },
                        "intensity": intensity,
                        "enabled": True
                    }
                })

            # Set the last interval as an unlabeled interval if the total count is odd
            if len(intervals) % 2 == 1:
                class_data["intervals"].pop()

        # Append the class structure to the classes list
        json_data["classes"].append(class_structure)

    # Write the JSON data to a file
    filename = protocol_name + '.json'
    with open(filename, 'w') as json_file:
        json.dump(json_data, json_file, indent=2)
    print ('Please set protocol parameters, preamble, epilogue, and groups before uploading your protocol')



In [2]:
#generate gesture protocol example
randomClassOrder = False
disconnectDeviceOnFinish = False
example_csv = 'protocol_example.csv'
generate_protocol(example_csv, "example_protocol", 0)

NameError: name 'name' is not defined

In [3]:
#generate Reaction Time protocol example
randomClassOrder = False
disconnectDeviceOnFinish = False
example_csv = 'rt_protocol_example.csv' 
generate_protocol(example_csv, "example_RT_protocol", 1)

In [1]:
randomClassOrder = False #Set to True if you want your classes to be random
disconnectDeviceOnFinish = False #Set to True if you want the device to disconnect after the protocol is finished
protocol_type = int(input('protocol type: 0 = for gestures 1 = for reaction time test   '))
dir_to_csv = input('dir to protocol csv   ')
generate_protocol(dir_to_csv, "your_protocol_name", protocol_type)